In [1]:
import os
import tensorflow as tf
from PIL import Image
import numpy as np
import random


# 원본 이미지 폴더 경로와 저장할 폴더 경로
input_folder = "../누끼/horse-or-human/train/horses"
output_folder_gray = "./dataset/horse-or-human/pre/horses"
output_folder_distort = "./dataset/horse-or-human/pre/horses"

# 저장할 폴더가 없으면 생성
os.makedirs(output_folder_gray, exist_ok=True)
os.makedirs(output_folder_distort, exist_ok=True)

# 이미지 크기를 유지하는 리사이즈 함수
def resize_keep_aspect_ratio(image, target_size):
    return tf.image.resize(image, target_size)

# 회색조 변환 함수 (스케일 값을 입력으로 받아 조정)
def convert_to_grayscale(image):
    # RGBA 형식을 RGB로 변환
    if image.shape[-1] == 4:
        image = image[:, :, :3]
    
    # 회색조 변환
    gray_image = tf.image.rgb_to_grayscale(image)
    return gray_image

# 랜덤 찌그러트림(왜곡) 함수 (Affine 변환)
def distort_image(image, shear_value):
    # 이미지의 크기 및 변환 행렬 설정
    image_height, image_width = tf.shape(image)[0], tf.shape(image)[1]
    
    # 찌그러트림 방향을 다양화 (X축, Y축에 각각 다른 랜덤 값 적용)
    shear_x = random.uniform(-shear_value, shear_value)  # X축 랜덤 찌그러트림
    shear_y = random.uniform(-shear_value, shear_value)  # Y축 랜덤 찌그러트림
    
    # Affine 변환 행렬
    transform_matrix = [1.0, shear_x, 0,    # X 축에 대한 변환
                        shear_y, 1.0, 0,    # Y 축에 대한 변환
                        0, 0]               # 이동 없음

    transform_matrix = tf.convert_to_tensor(transform_matrix, dtype=tf.float32)
    transform_matrix = tf.reshape(transform_matrix, [8])

    # 변환 적용
    distorted_image = tf.raw_ops.ImageProjectiveTransformV2(
        images=tf.expand_dims(image, 0),
        transforms=tf.expand_dims(transform_matrix, 0),
        output_shape=[image_height, image_width],
        interpolation="BILINEAR"
    )

    return tf.squeeze(distorted_image)

# 이미지 처리 함수
def process_images(input_folder, output_folder_gray, output_folder_distort):
    for image_name in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_name)
        if image_path.endswith(".jpg") or image_path.endswith(".png"):
            # 이미지 열기
            image = Image.open(image_path)
            image = np.array(image)

            # 이미지 Tensor로 변환
            image_tensor = tf.convert_to_tensor(image, dtype=tf.float32)

            # 원본 이미지 크기 유지
            original_size = tf.shape(image_tensor)[:2]

            # 1. 회색조 변환: 스케일 값을 0.5에서 1.0까지 0.1 단위로 조정
            # 회색조 변환
            gray_image = convert_to_grayscale(image_tensor)
            gray_image_np = gray_image.numpy().astype(np.uint8)
            gray_image_pil = Image.fromarray(gray_image_np.squeeze(), mode='L')  # PIL 이미지로 변환

            # 파일명에 "_grayscale" 접미사를 추가하여 저장
            file_name, file_extension = os.path.splitext(image_name)
            gray_image_pil.save(os.path.join(output_folder_gray, f"{file_name}_grayscale{file_extension}"))

            # 찌그러트림을 랜덤하게 적용하는 부분
            shear_value = random.uniform(0.1, 0.4)  # 0.1~0.4 사이의 랜덤 값 선택
            distort_image_tensor = distort_image(image_tensor, shear_value)
            distort_image_resized = resize_keep_aspect_ratio(distort_image_tensor, original_size)
            distort_image_np = distort_image_resized.numpy().astype(np.uint8)
            distort_image_pil = Image.fromarray(distort_image_np)

            # 파일명을 찌그러트림 값과 함께 저장
            distort_image_pil.save(os.path.join(output_folder_distort, f"distort_{shear_value:.1f}_{image_name}"))

# 이미지 처리 실행
process_images(input_folder, output_folder_gray, output_folder_distort)

print("이미지 전처리가 완료되었습니다!")

KeyboardInterrupt: 